### libraries

In [5]:
import pandas as pd
import numpy as np
import csv

### Anime Tupe Rules

In [10]:
class AnimeRules:
  def __init__(self, dirty_dataset, cleaned_dataset):
    self.dirty_dataset = dirty_dataset
    self.cleaned_dataset = cleaned_dataset
    self.df_d = pd.read_csv(dirty_dataset)
    
  def movie_rule(self):
    
        cnt = 0
        for index, row in self.df_d.iterrows():
            if float(row['episodes']) == 1.0 and ("movie" not in row['type'] or pd.isnull(row['type'])):
                    cnt+=1
                            
        print("Number of Detected Tuples are {}".format(cnt))

  def series_rule(self):
        cnt = 0
        for index, row in self.df_d.iterrows():
            if float(row['episodes']) != 1.0 and ("movie" in row['type'] or pd.isnull(row['type'])):
                    cnt+=1
                            
        print("Number of Detected Tuples are {}".format(cnt))
        
  def repairing(self):
        l = []
        for index, row in self.df_d.iterrows():
            if   float(row['episodes']) != 1.0 and ("movie" in row['type'] or pd.isnull(row['type'])):
                    row['type']="tv series"
            else:
                if float(row['episodes']) == 1.0 and ("movie" not in row['type'] or pd.isnull(row['type'])):
                    row['type']="movie"
                
            l.append(row)  

        df=pd.DataFrame(l)
        df.to_csv(self.cleaned_dataset, index = False)

### Error Detection & Repair

In [11]:
dirty_anime_data = "data/dirtyTrainDataset.csv"
cleaned_anime_data = "data/CleanedData.csv"

FixData = AnimeRules(dirty_anime_data, cleaned_anime_data)

print("Error detection of anime series")
FixData.series_rule()

print("Error detection of anime movies")
FixData.movie_rule()

print("repairing.....! check please data directory")
FixData.repairing()

Error detection of anime series
Number of Detected Tuples are 460
Error detection of anime movies
Number of Detected Tuples are 365
repairing.....! check please data directory


### Precision and Recall

In [4]:
original_data = pd.read_csv("/Users/sezin/PycharmProjects/dsa-DC4ML/Error Detection & Repair/Random Active Domain/TrainDataset.csv")
dirty_data = pd.read_csv("/Users/sezin/PycharmProjects/dsa-DC4ML/Error Detection & Repair/Random Active Domain/dirtyTrainDataset.csv")
cleaned_data = pd.read_csv("/Users/sezin/PycharmProjects/dsa-DC4ML/Error Detection & Repair/Random Active Domain/CleanedData.csv")

matrix = pd.DataFrame(columns=["predicted True", "predicted False"], index=["actual True", "actual False"], data=[[0,0],[0,0]])

print(original_data.head())
print(dirty_data.head())
print(cleaned_data.head())

for column in ["type"]:
    for i in range(0, len(cleaned_data[column])):
        
        # 1 - Cleaned in correct way 
        if cleaned_data[column][i] == original_data[column][i] and original_data[column][i] != dirty_data[column][i]:
            matrix["predicted True"]["actual True"] = matrix["predicted True"]["actual True"]+1
            #print("Detected error and fixed")
            
        # 2 - Cleaned in wrong way: Detected error and replaced wrong
        elif cleaned_data[column][i] != original_data[column][i] and original_data[column][i] != dirty_data[column][i]:
            matrix["predicted False"]["actual True"] = matrix["predicted False"]["actual True"]+1
            #print("Did not find error")
            
        # No cleaning necessary - no cleaning done
        elif cleaned_data[column][i] == original_data[column][i] and original_data[column][i] == dirty_data[column][i]:
            matrix["predicted False"]["actual False"] = matrix["predicted False"]["actual False"]+1
            #print("No error implemnted - nothing fixed")
            
        # Cleaned - no cleaning necessary
        elif cleaned_data[column][i] != original_data[column][i] and original_data[column][i] == dirty_data[column][i]:
            matrix["predicted True"]["actual False"] = matrix["predicted True"]["actual False"]+1
            #print("No error implemented - fixed wrong")
            
print("Detected error and fixed",matrix["predicted True"]["actual True"])
print("Did not find error",matrix["predicted False"]["actual True"])
print("No error implemnted - nothing fixed",matrix["predicted False"]["actual False"])
print("No error implemented - fixed wrong",matrix["predicted True"]["actual False"])



   anime_id                               name  \
0      9274            Tiger &amp  Bunny Pilot   
1      5089                      Stay the Same   
2     33546        Rennyuu Tales The Animation   
3     24071                    High School DxD   
4     17707  Huang Feihong Yong Chuang Tianxia   

                                     genre       type  episodes  rating  \
0                 Action,Comedy,SuperPower      movie       1.0    6.05   
1              Documentary,Short,Biography  tv series      13.0    6.48   
2                                   Hentai      movie       1.0    7.00   
3  Animation,Action,Comedy,Fantasy,Romance      movie       1.0    6.71   
4               Action,MartialArts,Shounen  tv series       3.0    5.74   

   members                                           overview  
0      389                                                NaN  
1      156  'Recorded every day for a year at exactly the ...  
2      854  "Lupin comes into the possession of half a d